In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
data_path = "./dane/"
df = pd.read_excel(os.path.join(data_path, "dane1.xlsx"))
df.describe()

,Zadanie,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10
count,50.00000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,25.50000,55.060000,48.520000,51.300000,47.900000,50.740000,49.500000,49.760000,49.400000,52.880000,53.660000
std,14.57738,28.287453,26.560858,29.768666,26.896817,29.139222,29.532583,28.668094,27.913862,26.106251,28.688766
min,1.00000,4.000000,6.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,2.000000,2.000000
25%,13.25000,37.000000,28.750000,30.250000,27.500000,23.750000,26.250000,28.500000,29.000000,37.250000,30.250000
50%,25.50000,55.000000,49.000000,53.500000,48.500000,50.500000,48.500000,46.000000,54.500000,54.000000,59.000000
75%,37.75000,78.750000,67.750000,75.250000,68.750000,71.000000,71.000000,73.250000,72.750000,73.250000,77.500000
max,50.00000,96.000000,98.000000,99.000000,99.000000,99.000000,99.000000,97.000000,94.000000,98.000000,99.000000


In [3]:
# wartość optymalizacji po poszczególnym ruchu
optimalization = pd.DataFrame(columns=["Wartosc kombinacji"])
Neighbours_count = 20
improvements = np.zeros(Neighbours_count)
df_backup=df

In [4]:
def E(df=df):
    if len(df) <= 0:
        return 0
    elif len(df) == 1:
        return int(df.sum(axis=1))

    obrobione = df.copy()
    obrobione["M1"] = df["M1"].cumsum() # TODO
    obrobione.iloc[1] = df.iloc[1].cumsum()
    
    for i in range(2, len(obrobione)):
        for j in range(2, len(obrobione.iloc[1])):
            max_val = max(int(obrobione.iloc[[i-1], [j]].values),
                             int(obrobione.iloc[[i], [j-1]].values))
            obrobione.iloc[[i], [j]] = max_val + df.iloc[[i], [j]]
    wartosc = int(obrobione.iloc[[-1], [-1]].values)
    
    return wartosc


In [5]:
# KR to wartość naszego rozwiązania - ma być najmniejsza
#def E(df = df):
 #   c_sum = df["Czas wykonania"].cumsum() # suma kumultatywna kolumny "Czas wykonania"
  #  KR = (df["Termin"] - c_sum)**2 # obliczanie kwadratu różnicy wartosći
   # return sum(KR) # KR jest wskaźnikiem jakości ułożenia zadań. To optymalizujemy

# wyszukiwanie listy sąsiadów
def get_neighbours(idx, neighbours_count = 20, df = df):
    N = neighbours_count
    # Idea sąsiedztwa dla N = 6 i idx = n E <N/2 ; L-N/2>
    # idx = X, sąsiad = 0, nie sąsiad = -
    # nr_indeksu | ... | n-4 | n-3 | n-2 | n-1 |  n  | n+1 | n+2 | n+3 | n+4 | ... |
    # sasiedztwo |  -  |  -  |  0  |  0  |  0  |  X  |  0  |  0  |  0  |  -  |  -  |
    
    L = len(df) - 1 
    idx_prev = idx # indeks dla ktorego szukamy sasiadow
    N_half = N//2 # polowa wartosci N
    
    # Idea sąsiedztwa dla N = 6 i idx = n < N/2
    # idx = X, sąsiad = 0, nie sąsiad = -
    # przykład n = 2
    # nr_indeksu |  0  |  1  | n=2 |  3  |  4  |  5  |  6  |  7  |  8  |  9  | ... |
    # sasiedztwo |  0  |  0  |  X  |  0  |  0  |  0  |  0  |  -  |  -  |  -  |  -  |
    # skoro nie mozemy znaleźć wystarczająco dużo sąsiadów po lewej, 
    # szukamy dodatkowego po prawej
    if idx < N_half:
        idx = N_half
    # idea sąsiedztwa z ograniczeniem z prawej jest analogiczna do tej z ograniczeniem z lewej
    elif idx > L - N_half:
        idx = L - N_half
    output = list(range(idx - N_half, idx + N_half + 1))
    # usuwanie idx z output
    # TODO: na pewno da się to zrobić prościej
    rm_idx = output.index(idx_prev)
    output = np.array(output)
    output = np.delete(output, rm_idx)
    return output

def swap(idx_1, idx_2, df=df):
    # zamiana miejscami dwóch wierszy "in place" - nie tworzymy nowej ramki danych
    df.iloc[idx_1], df.iloc[idx_2] = df.iloc[idx_2].copy(), df.iloc[idx_1].copy()
    
def move(idx_1, idx_2, df=df):
    swap(idx_1, idx_2) # ruch

def reduce_temperature(T, rate_of_change = .02, sposob = 1):
    return (1-rate_of_change)*T if sposob == 1 else T/(1+rate_of_change*T)

def plot_optimalization(optimalization):
    # TODO: można to zrobić ładniej matplotlibem
    optimalization["Wartosc kombinacji"].plot(kind="line")



In [6]:
def simulated_anniling_algorithm(
    T = 2500, 
    Tmin = 100, 
    neighbours_count=20, 
    number_of_generations = 1000
):
    global df
    R = np.random.randint(len(df)) # losowy numer wiersza
    KR_before = E()
    optimalization = pd.DataFrame(columns=["Wartosc kombinacji"])
    optimalization = optimalization.append({
                    "Wartosc kombinacji": KR_before
                }, ignore_index=True)
    while T > Tmin:
        while number_of_generations:
            print(f"{number_of_generations} iterations left")
            neighbours = get_neighbours(idx=R, neighbours_count=neighbours_count) # generuj sąsiadów
            R_prim = np.random.choice(neighbours) # losowy indeks wiersza
            move(R, R_prim)
            KR_now = E()
            dE = KR_now - KR_before
                        # prawdopodobieństwo przyjęcia gorszego rozwiązania
            if dE < 0 or np.random.rand() < np.exp(-dE/T):
                R = R_prim
            KR_before = KR_now
            # dodawanie obecnej wartości kombinacji do listy wszystkich wartości
            optimalization = optimalization.append({
                    "Wartosc kombinacji": KR_now
                }, ignore_index=True)
            number_of_generations -= 1
        T = reduce_temperature(T)
    order = df["Zadanie"]
    return order, optimalization
            

In [ ]:
if __name__ == '__main__':
    new_order, new_optimalization = simulated_anniling_algorithm(
        T=2500,
        Tmin=100,
        neighbours_count=20,
        number_of_generations=150
    )
    plot_optimalization(new_optimalization)
    new_optimalization["Wartosc kombinacji"].iloc[-1]

150 iterations left
149 iterations left
148 iterations left
147 iterations left
146 iterations left
145 iterations left
144 iterations left
143 iterations left
142 iterations left
141 iterations left
140 iterations left
139 iterations left
138 iterations left
137 iterations left
136 iterations left
135 iterations left
134 iterations left
133 iterations left
132 iterations left
131 iterations left
130 iterations left
129 iterations left
128 iterations left
127 iterations left
126 iterations left
125 iterations left
124 iterations left
123 iterations left
122 iterations left
121 iterations left
120 iterations left
119 iterations left
118 iterations left
117 iterations left


In [ ]:
plot_optimalization(new_optimalization)

In [ ]:
for order in new_order:
    print(order)

In [ ]:
#pd.to_csv()
order

In [ ]:
df.to_csv("./Multistart Results/wynik_wyzazanie2dane1.csv")